In [1]:
import sys
import os
import random
import numpy as np
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

import random
import numpy as np


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
    #cudnn.benchmark = True

# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST的均值和标准差
])

# 加载数据集
kwargs = {'num_workers': 8, 'pin_memory': True}
train_dataset = datasets.MNIST(root='/chenyupeng/old_files/yupeng_landscape/landscape_LLM/MNIST_MLP/data2', 
                              train=True, 
                              transform=transform,
                              download=True)
test_dataset = datasets.MNIST(root='/chenyupeng/old_files/yupeng_landscape/landscape_LLM/MNIST_MLP/data2', 
                             train=False, 
                             transform=transform)

# 创建数据加载器
train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=60000, 
                         shuffle=False,**kwargs)
test_loader = DataLoader(dataset=test_dataset, 
                        batch_size=60000, 
                        shuffle=False,**kwargs)
input_size = 784  # 28x28
hidden_size =  5
output_size = 10  # 0-9的数字

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:02<00:00, 3470200.42it/s]


Extracting /chenyupeng/old_files/yupeng_landscape/landscape_LLM/MNIST_MLP/data2/MNIST/raw/train-images-idx3-ubyte.gz to /chenyupeng/old_files/yupeng_landscape/landscape_LLM/MNIST_MLP/data2/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 101261.08it/s]


Extracting /chenyupeng/old_files/yupeng_landscape/landscape_LLM/MNIST_MLP/data2/MNIST/raw/train-labels-idx1-ubyte.gz to /chenyupeng/old_files/yupeng_landscape/landscape_LLM/MNIST_MLP/data2/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:01<00:00, 936071.31it/s] 


Extracting /chenyupeng/old_files/yupeng_landscape/landscape_LLM/MNIST_MLP/data2/MNIST/raw/t10k-images-idx3-ubyte.gz to /chenyupeng/old_files/yupeng_landscape/landscape_LLM/MNIST_MLP/data2/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 6693790.85it/s]


Extracting /chenyupeng/old_files/yupeng_landscape/landscape_LLM/MNIST_MLP/data2/MNIST/raw/t10k-labels-idx1-ubyte.gz to /chenyupeng/old_files/yupeng_landscape/landscape_LLM/MNIST_MLP/data2/MNIST/raw



In [4]:
len(train_dataset)

60000

In [5]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size,bias=False)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size,bias=False)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [6]:
model = MLP(input_size, hidden_size, output_size)

eval_batches = train_loader # 使用前10个batch评估


criterion = nn.CrossEntropyLoss()

In [7]:
def eval_loss(datalodaer, model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        running_loss = 0.0
        for images, labels in datalodaer:
            labels = labels.cuda()
            images = images.reshape(-1, 28 * 28).cuda()
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_loss += loss.item()

    return running_loss / len(datalodaer), 100 * correct / total

In [8]:
def eval_gradient(datalodaer, model):
    model.eval()
    #with torch.no_grad():
    correct = 0
    total = 0
    running_loss = 0.0
    for images, labels in datalodaer:
        labels = labels.cuda()
        images = images.reshape(-1, 28 * 28).cuda()
        outputs = model(images)
        loss = criterion(outputs, labels)

        scaled_loss = loss / len(datalodaer)
        scaled_loss.backward()  # 梯度会累积

    
    grad_norm = 0
    for p in model.parameters():
        if p.grad is not None:
            #p.data -= lr * p.grad
            grad_norm += (p.grad.norm())**2
            p.grad = None  # 清空梯度
            
    grad_norm = torch.sqrt(grad_norm)
    
    return grad_norm

In [9]:
import time
def get_full_hessian(datalodaer, model):
    model.eval()
    model.zero_grad(set_to_none = True)
    def hessian_calculation(g_tensor):
        g_tensor = g_tensor.cuda()
        total_params = g_tensor.size(0)
        hessian_list = []
        t_d = time.time()
        for d in range(total_params):
            unit_vector = torch.zeros(total_params)
            unit_vector[d] = 1
            unit_vector = unit_vector.cuda()
            l = torch.sum(g_tensor * unit_vector)
            l.backward(retain_graph= True)
            hessian_row = []
            for name, param in model.named_parameters():
                if 'ln' in name or 'bias' in name or 'wte' in name or 'wpe' in name:
                    continue
                if param.requires_grad:
                    #print('name',name, param.grad)
                    hessian_row.append(param.grad.double().data.clone())
            
            model.zero_grad(set_to_none = True)
            hessian_row = [g.flatten() for g in hessian_row] 
            hessian_row = [g.cpu() for g in hessian_row]
            hessian_row = torch.cat(hessian_row)
            #print('hessian_row', hessian_row)   
            hessian_list.append(hessian_row)
            # if d % 1000 == 0:
            #     print(f'Computing hessian: current batch = {batch_idx}/{self.num_batches}, current row of a hessian: {d}/{total_params}, total time = {time.time()- t_d} ')
        hessian = torch.stack(hessian_list, dim = 1)
        #print('hessian', hessian)   
        return hessian
    full_hessian = 0
    for images, labels in datalodaer:
        labels = labels.cuda()
        images = images.reshape(-1, 28 * 28).cuda()
        outputs = model(images)
        loss = criterion(outputs, labels)

        scaled_loss = loss / len(datalodaer)
        scaled_loss.backward(create_graph= True)
        g_list = []
        count = 0
        for name, param in model.named_parameters():
            #if 'ln' in name or 'bias' in name:
            if 'ln' in name or 'bias' in name or 'wte' in name or 'wpe' in name:
                continue
            if param.requires_grad:
                count += param.numel()
                #print('g shape', param.grad , param.grad.shape)
                g_list.append(torch.flatten(param.grad.double()))
                #print('name',name, g_list[-1].size())
        g_tensor = torch.cat(g_list, dim = 0)

        #print(g_tensor.shape)
        #print('g_tensor',g_tensor)
        model.zero_grad(set_to_none = True)
        H = hessian_calculation(g_tensor)
        full_hessian += H
    full_hessian = torch.nan_to_num(full_hessian, nan = 0, posinf = 0, neginf = 0 )  # change nan, postive inf , negative inf, to 0
    t_svd = time.time()
    #print('doing EVD')
    # _, eigenvalues, _ = torch.linalg.svd(full_hessian)  # ascending
    #eigenvalues, _  = torch.eig(full_hessian)
    full_hessian = full_hessian.numpy().astype(np.float64)
    full_hessian = (full_hessian + full_hessian.T)/2 # make symetric, to 
    
    
    
    #avoid numerical issue
    #full_hessian = full_hessian.cuda()
    #eigenvalues, _  = torch.linalg.eig(full_hessian)
    # eigenvalues, _  = np.linalg.eigh(full_hessian)
    # #_, eigenvalues, _ = np.linalg.svd(full_hessian) 
    # eigenvalues = [eigen.item().real for eigen in eigenvalues]
    # file_name = self.file_dir + 'eigenvalues.txt'
    # with open(file_name, "w") as file:
    #     for item in eigenvalues:
    #         file.write(str(item)+"\n")
    # print(f'EVD time = {time.time()- t_svd}')
    return full_hessian, g_tensor

In [10]:
import os

script_dir = os.getcwd()
ckpt_dir= "bs256_lr0.05_e30_hidden5_SGD_m0.9_seed0"
ckpt_name = [i*0.1 for i in range(1,200)]
ckpt_files = [f"{script_dir}/trained_MLP/{ckpt_dir}/model_para_e_{i:.1f}_decaying_2.0_pipecheck_w.pt" for i in ckpt_name]

all_decay_model = {}

for name, ckpt in zip(ckpt_name, ckpt_files):
    name_str = f"{name:.1f}"
    temp_model = MLP(28*28, 5, 10)
    state_dict = torch.load(ckpt, map_location=torch.device('cpu'))
        ## 兼容 state_dict 可能存在 'model' key 的情况
        #if 'model_state_dict' in state_dict:
    temp_model.load_state_dict(state_dict['state_dict'])
    all_decay_model.update({name_str: temp_model})

In [11]:
hessian_0, grad = get_full_hessian(eval_batches, all_decay_model['0.1'].cuda())

/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:266: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at ../torch/csrc/autograd/engine.cpp:1177.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [12]:
grad.shape

torch.Size([3970])

In [13]:
model

MLP(
  (fc1): Linear(in_features=784, out_features=5, bias=False)
  (relu): ReLU()
  (fc2): Linear(in_features=5, out_features=10, bias=False)
)

In [14]:
hessian_0 = torch.tensor(hessian_0)

u,sigma,d = torch.svd(hessian_0.cuda())

In [15]:
sigma

tensor([1.7800e+01, 1.4120e+01, 1.2046e+01,  ..., 5.9897e-37, 3.8653e-37,
        3.7000e-37], device='cuda:0', dtype=torch.float64)

In [16]:
torch.matmul(grad.reshape(-1,1).T, torch.matmul(hessian_0.cuda(), grad.reshape(-1,1)))/torch.norm(grad)**2

tensor([[11.0027]], device='cuda:0', dtype=torch.float64,
       grad_fn=<DivBackward0>)

In [17]:
hessian_0, grad = get_full_hessian(eval_batches, all_decay_model['1.0'].cuda())
hessian_0 = torch.tensor(hessian_0)

u,sigma,d = torch.svd(hessian_0.cuda())
torch.matmul(grad.reshape(-1,1).T, torch.matmul(hessian_0.cuda(), grad.reshape(-1,1)))/torch.norm(grad)**2

tensor([[7.9624]], device='cuda:0', dtype=torch.float64,
       grad_fn=<DivBackward0>)

In [18]:
hessian_0, grad = get_full_hessian(eval_batches, all_decay_model['19.0'].cuda())
hessian_0 = torch.tensor(hessian_0)

u,sigma,d = torch.svd(hessian_0.cuda())
torch.matmul(grad.reshape(-1,1).T, torch.matmul(hessian_0.cuda(), grad.reshape(-1,1)))/torch.norm(grad)**2

tensor([[7.7162]], device='cuda:0', dtype=torch.float64,
       grad_fn=<DivBackward0>)

In [ ]:
for v in all_decay_model.values():
    print(f"gradient norm : {eval_gradient(eval_batches, v.cuda())}")

gradient norm : 0.16438806056976318
gradient norm : 0.1482352763414383
gradient norm : 0.1415596306324005
gradient norm : 0.13238200545310974
gradient norm : 0.11943792551755905
gradient norm : 0.11839990317821503
gradient norm : 0.11563529819250107
gradient norm : 0.11019901931285858
gradient norm : 0.10939928889274597
gradient norm : 0.1074424460530281
gradient norm : 0.10870210826396942
gradient norm : 0.10800721496343613
gradient norm : 0.10809173434972763
gradient norm : 0.10720718652009964
gradient norm : 0.11178351938724518
gradient norm : 0.10957272350788116
gradient norm : 0.10929979383945465
gradient norm : 0.10949341952800751
gradient norm : 0.10991955548524857
gradient norm : 0.1070091649889946
gradient norm : 0.11140109598636627
gradient norm : 0.11072598397731781
gradient norm : 0.11037047207355499
gradient norm : 0.10863151401281357
gradient norm : 0.10888320952653885
gradient norm : 0.11008898913860321
gradient norm : 0.1091446802020073
gradient norm : 0.112929083406925